In [1]:
from bilstm_crf import BiLSTMCRF
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Load material

In [2]:
import json

# load embedding
embedding_maxtrix = np.load('embedding/embedding_matrix.npy')

# load vocab
with open('data/vocab.txt', 'r') as f:
    vocab = f.read().split('\n')
len(vocab)

# load tag_to_id
with open('data/tag_to_id.json', 'r') as f:
    tag_to_id = json.load((f))

# load train and dev data
TRAIN_PATH = 'data/span_detection_datasets_IOB/train.json'
DEV_PATH = 'data/span_detection_datasets_IOB/dev.json'

with open(TRAIN_PATH, 'r') as f:
    train_data = json.load(f)

with open(DEV_PATH, 'r') as f:
    dev_data = json.load(f)

train_sentences = list(train_data['text'].values())
dev_sentences = list(dev_data['text'].values())

train_labels = list(train_data['labels'].values())
dev_labels = list(dev_data['labels'].values())

# Convert data

## Sentences

In [3]:
import numpy as np

# Convert data to ids
def convert_to_ids(data, vocab, max_len=256):
    id_data = []

    pad_token_id = vocab.index('<PAD>')
    ukn_token_id = vocab.index('<UNK>')
    for sentence in data:
        ids = []
        for word in sentence.split():
            if word in vocab:
                ids.append(vocab.index(word))
            else:
                ids.append(ukn_token_id)

        if len(ids) < max_len:
            ids += [pad_token_id] * (max_len - len(ids))
        id_data.append(np.array(ids))
        
    return id_data

In [4]:
train_tokenized = convert_to_ids(train_sentences, vocab)
dev_tokenized = convert_to_ids(dev_sentences, vocab)

train_tokenized = [torch.LongTensor(tokenized) for tokenized in train_tokenized]
dev_tokenized = [torch.LongTensor(tokenized) for tokenized in dev_tokenized]

## Labels

In [5]:
# load tag_to_id
with open('data/tag_to_id.json', 'r') as f:
    tag_to_id = json.load((f))

# Convert labels to ids
# labels = [[start, end, tag], ...]
def convert_labels_to_ids(label, tag_to_id, max_len=256):
    ids = [tag_to_id[tag] for tag in label]

    if len(ids) < max_len:
        ids += [tag_to_id['O']] * (max_len - len(ids))
        
    return np.array(ids)

In [6]:
train_labels_encoding = np.array([convert_labels_to_ids(label, tag_to_id) for label in train_labels])
dev_labels_encoding = np.array([convert_labels_to_ids(label, tag_to_id) for label in dev_labels])

train_labels_encoding = np.expand_dims(train_labels_encoding, axis=2)
dev_labels_encoding = np.expand_dims(dev_labels_encoding, axis=2)

train_labels_encoding = [torch.LongTensor(label) for label in train_labels_encoding]
dev_labels_encoding = [torch.LongTensor(label) for label in dev_labels_encoding]

# Model

In [7]:
span_detection_model = BiLSTMCRF(vocab_size=len(vocab), tag_to_ix=tag_to_id, hidden_dim=200, embedding_maxtrix=embedding_maxtrix)

In [8]:
# train
optimizer = optim.SGD(span_detection_model.parameters(), lr=0.01, weight_decay=1e-4)
epoch_num = 10

for epoch in range(epoch_num):
    print('epoch: {}'.format(epoch))
    for i in range(len(train_tokenized)):
        sentence = train_tokenized[i]
        label = train_labels_encoding[i]
        span_detection_model.zero_grad()
        loss = span_detection_model.neg_log_likelihood(sentence, label)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print('loss: {}'.format(loss.item()))

    # dev test
    # with torch.no_grad():
    #     for i in range(len(dev_sentences)):
    #         sentence = dev_sentences[i]
    #         label = dev_labels_encoding[i]
    #         span_detection_model.zero_grad()
    #         loss = span_detection_model.neg_log_likelihood(sentence, label)
    #         loss.backward()
    #         optimizer.step()
    #         if i % 100 == 0:
    #             print('loss: {}'.format(loss.item()))

# save model
# torch.save(span_detection_model.state_dict(), 'model/span_detection_model.pkl')

epoch: 0
forward_score:  tensor(-789.1182, grad_fn=<SumBackward0>)


c:\Users\Hii\miniconda3\envs\udemy\Lib\site-packages\TorchCRF\__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ..\aten\src\ATen\native\TensorCompare.cpp:493.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


gold_score:  tensor([-789.1182], grad_fn=<SubBackward0>)
loss: 0.0
forward_score:  tensor(-790.2479, grad_fn=<SumBackward0>)
gold_score:  tensor([-790.2479], grad_fn=<SubBackward0>)
forward_score:  tensor(-788.3355, grad_fn=<SumBackward0>)
gold_score:  tensor([-788.3355], grad_fn=<SubBackward0>)
forward_score:  tensor(-789.6654, grad_fn=<SumBackward0>)
gold_score:  tensor([-789.6654], grad_fn=<SubBackward0>)
forward_score:  tensor(-790.9197, grad_fn=<SumBackward0>)
gold_score:  tensor([-790.9197], grad_fn=<SubBackward0>)
forward_score:  tensor(-789.2810, grad_fn=<SumBackward0>)
gold_score:  tensor([-789.2810], grad_fn=<SubBackward0>)
forward_score:  tensor(-789.6131, grad_fn=<SumBackward0>)
gold_score:  tensor([-789.6131], grad_fn=<SubBackward0>)
forward_score:  tensor(-788.9012, grad_fn=<SumBackward0>)
gold_score:  tensor([-788.9012], grad_fn=<SubBackward0>)
forward_score:  tensor(-791.0895, grad_fn=<SumBackward0>)
gold_score:  tensor([-791.0895], grad_fn=<SubBackward0>)
forward_score

KeyboardInterrupt: 

# End